In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.figure12 import Figure12
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# env = Figure12(5, 5, (4, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [3]:
# adam_and_eve = []
# env.reset()
# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# # init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# # init_learner = Learner(uuid.uuid4(), init_rule)
# # (init_team.learners).append(init_learner)
# # init_team.mostRecent = init_learner
# adam_and_eve.append(init_team)

# for adam in adam_and_eve:
#     print(adam.learners)
#     print('------')
#     adam.init_search(env, [2, 3])
#     print(adam.learners)
#     for learner in adam.learners:
#         print(learner.program.rule.action_set)
#         print(learner.program.rule.region)

In [4]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 1, 1, 0)
init_team.init_search(env, [2, 3])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
#     win = adam.search(env)
    data = [adam, win]
    searchManager.evaluate_team(data)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    if win:
        print(f'got to --> gen {gen+1}')
        break
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
#     if gen == 0:
#         for adam in adam_and_eve:
#             win = adam.search(env)
#             data = [adam, win]
#             searchManager.evaluate_team(data)
#     else:
    for parent in searchManager.teamPool:
        # only run if the team has not yet won
        if parent not in searchManager.winners:
            print(f'Parent: {parent.id}')
            env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search(env)
            print(f'Child: {child.id}')
            print('Start Child Regions --------')
            for learner in child.learners:
                print(learner.program.rule.region)
            print('End Child Regions --------')
            print(f'Child Fitness: {child.fitness}') 
            toEvaluateAfterGen.append([child, win])
        else:
            print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

new step-------
3
(0, 0)
new step-------
3
(0, 0)
new step-------
2
(0, 1)
new step-------
2
(0, 2)
new step-------
2
(0, 3)
new step-------
2
(0, 4)
new step-------
2
(0, 4)
[0, 0, 0, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: ef2abdf9-76b8-4ca7-8c0c-1f65573d4a16
Start: (0, 4)
New real step: 
1
(0, 4)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 0, 4]
New real step: 
1
(0, 1)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
Child: 2892838c-c3f5-40f9-837c-3278a1eabce9
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
End Child Regions --------
Child Fitness: 0.30000000000000004
%%%

3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
Child: 3f3bcd3f-a6fc-4068-9b35-cf4d8a41d9f4
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
End Child Regions --------
Child Fitness: 5.01
Parent: f29693ef-d7a6-4d8f-874a-6eafee4da121
Start: (4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3)
New real step: 
3
(4, 2)
New real step: 
3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
Child: 89b6128c-d709-4f33-a890-8bd033e985d6
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
End Child Regions --------
Child Fitness: 5.01
Parent: c1c9b7c2-030a-42cc-9332-66bbd53208b5
Start: (4, 4)
New real step: 
0
(4, 4)
Backtrackregioncehck: [0, 4, 0, 3]
New real step: 
0
(4, 2)
Backtrackreg

End Child Regions --------
Child Fitness: 6.97
Parent: 1bdaf3d0-2112-4a14-a696-3f668a6d5da5
Start: (4, 0)
New real step: 
0
(4, 0)
Backtrackregioncehck: [0, 4, 1, 4]
New real step: 
1
(3, 2)
New real step: 
1
(2, 2)
New real step: 
1
(1, 2)
New real step: 
1
(0, 2)
New real step: 
1
(0, 2)
Child: 151938fc-5ce9-4630-abe4-7c15b847b4ce
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
End Child Regions --------
Child Fitness: 6.38
Parent: 7b2aba56-f477-4f08-b903-ee93e7dd138f
Start: (4, 2)
New real step: 
3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
New real step: 
2
(4, 1)
New real step: 
2
(4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
Child: 187b196b-c6b3-4419-abbe-26ace41423bd
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 

[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
Team: 4 --> 8.139999999999999
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
Team: 5 --> 7.949999999999999
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 19 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 20 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: f93aef03-dcff-424e-8ed8-3c7b989be35e
Start: (4, 0)
New real step: 
1
(3, 0)
New real step: 
1
(3, 0)
New real step: 
0
(4, 0)
New real step: 
0
(4, 0)
Child: 9c24ea83-c29f-4b04-a9c0-e045a18d20a5
Start Ch

2
(4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3)
New real step: 
3
(4, 2)
New real step: 
3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
Child: dfecfc48-07c8-4534-a8bf-bf3dabc314e3
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
End Child Regions --------
Child Fitness: 9.909999999999997
Parent: f89c34ef-c50f-4528-ab8e-cbba74f1ccac
Start: (0, 0)
New real step: 
0
(1, 0)
New real step: 
0
(1, 0)
New real step: 
1
(0, 0)
New real step: 
1
(0, 0)
Backtrackregioncehck: [0, 0, 1, 4]
New real step: 
1
(0, 2)
Backtrackregioncehck: [0, 0, 2, 4]
New real step: 
1
(0, 3)
Backtrackregioncehck: [0, 0, 3, 4]
New real step: 
1
(0, 4)
New real step: 
1
(0, 0)
Backtrackregioncehck:

[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
End Child Regions --------
Child Fitness: 11.799999999999999
Parent: de689019-50fb-4b2d-9748-912bfde423b3
Start: (0, 4)
New real step: 
0
(0, 4)
Backtrackregioncehck: [0, 0, 0, 3]
New real step: 
0
(1, 2)
New real step: 
0
(2, 2)
New real step: 
0
(3, 2)
New real step: 
0
(4, 2)
New real step: 
0
(4, 2)
Child: c96a021b-db2f-45ce-882d-66286cc4429d
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
End Child Regions --------
Child Fitness: 12.00000

Child: 945094cd-466c-4439-bbb9-d9edddc0494f
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 3, 4]
[1, 2, 1, 1]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 12.500000000000002
Parent: c0ed20e3-2861-409f-889e-f0846612a994
Start: (4, 2)
New real step: 
2
(4, 3)
New real step: 
2
(4, 4)
New real step: 
2
(4, 4)
New real step: 
3
(4, 3)
New real step: 
3
(4, 2)
New real step: 
3
(4, 1)
New real step: 
3
(4, 0)
New real step: 
3
(4, 0)
Child: 31cd395f-3d9b-4ac4-abbc-9dd33470bc84
Start Child Regions --------
[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[1, 2, 1, 4]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 1, 4]
[1, 0, 3, 3]
[0, 4, 3, 4]
[

In [5]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1002.2700000000001
Learners:
Region: [0, 0, 4, 4] --> Action: [2, 3]
Region: [1, 4, 0, 0] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 0, 3] --> Action: [2, 3]
Region: [1, 4, 2, 4] --> Action: [0, 1]
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1004.23
Learners:
Region: [0, 0, 4, 4] --> Action: [2, 3]
Region: [1, 4, 0, 0] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 3, 4] --> Action: [2, 3]
Region: [1, 2, 1, 4] --> Action: [0, 1]
Region: [0, 0, 0, 1] --> Action: [2, 3]
Region: [1, 2, 0, 3] --> Action: [0, 1]
Region: [0, 4, 0, 3] --> Action: [2, 3]
Region: [1, 4, 2, 4] --> Action: [0, 1]
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1004.23
Learners:
Region: [0, 0, 4, 4] --> Action: [2, 3]

In [7]:
team = searchManager.winners[0]
for learner in team.learners:
    print(learner.program.rule.region)

for i in range(5):
    print(f'Epoch: {i+1} --------')
    env.reset()
    win = False
    selected_learner = team.select_learner(env)
    while not win:
        win, needs_update, reward, winning_action = team.q_evaluation(env, selected_learner)
        print(reward)
        if not win:
            selected_learner = team.select_learner(env)
            team.transition_update(reward, winning_action, needs_update, selected_learner)

    team.final_update(reward, winning_action, needs_update)

print(win)
for learner in team.learners:
    print(f'{learner.program.rule.region}')
    for i in range(len(learner.program.rule.action_set)):
          print(f'{learner.program.rule.action_set[i]} --> {learner.program.rule.value_set[i]}')

[0, 0, 4, 4]
[1, 4, 0, 0]
[0, 0, 0, 1]
[1, 2, 0, 3]
[0, 4, 0, 3]
[1, 4, 2, 4]
Epoch: 1 --------
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
Winning action: 0
0.1
[0, 4, 0, 3]
Winning action: 2
0.1
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
Epoch: 2 --------
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
Winning action: 0
0.1
[0, 4, 0, 3]
Winning action: 2
0.1
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
Epoch: 3 --------
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
Winning action: 0
0.1
[0, 4, 0, 3]
Winning action: 2
0.1
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
Epoch: 4 --------
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
Winning action: 0
0.1
[0, 4, 0, 3]
Winning action: 2
0.1
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
Epoch: 5 --------
[0, 0, 0, 1]
Winning action: 2
0.1
[1, 2, 0, 3]
Winning action: 0
0.1
[0, 4, 0, 3]
Winning action: 2
0.1
[1, 4, 2, 4]
Winning action: 1
win!
1000.1
True
[0, 0, 4, 4]
2 --> 0
3 --> 0
[1, 4, 0, 0]
0 --> 0
1 --> 0
[0, 0, 0, 1]
2 --> 1000.40000